## Pull in UDFs

In [86]:
%run nb_udfs

StatementMeta(, ab14784b-9cc0-4e5b-a91a-b1b073f0fd01, 308, Finished, Available, Finished)

## Run Activities

In [87]:
workspace = 'Admin%20Center' #have to escape the & symbol and spaces
lakehouse = 'lh_monitoring'

act_table = 'factActivities'
date_offset = 1
#defaults the activity data to grab yesterday's data. The API only goes back 28 days maximum.


StatementMeta(, ab14784b-9cc0-4e5b-a91a-b1b073f0fd01, 309, Finished, Available, Finished)

In [88]:
"""
Gets the Power BI activity events for a specific day (offset from today).
:param date_offset: Number of days ago (0 = today, 1 = yesterday). Defaults to 1.
"""
# Calculate the UTC date string
target_date = (dt.utcnow() - timedelta(days=date_offset)).strftime('%Y-%m-%d')
start_date = f"{target_date}T00:00:00.00Z"
end_date = f"{target_date}T23:59:59.99Z"
print(f"🔍 Fetching activity for {start_date} to {end_date} - date offset {date_offset}")

StatementMeta(, ab14784b-9cc0-4e5b-a91a-b1b073f0fd01, 310, Finished, Available, Finished)

🔍 Fetching activity for 2025-09-10T00:00:00.00Z to 2025-09-10T23:59:59.99Z - date offset 28


In [89]:
# Call the API

windows = generate_hourly_windows(target_date)
#print(windows)

#test = list_activity_events('2025-04-19T00:00:00.000Z','2025-04-19T00:59:59.999Z')
#print(test)

# Define dimension load config
activity_path = udf_GetFilePath(workspace, lakehouse, act_table)
natural_key = ['Id']
primary_key = 'tableId'


for start_time, end_time in windows:
    print(f"Fetching events from {start_time} to {end_time}")
    df_activities = list_activity_events(start_time, end_time)
    #print(df_activities)
    if df_activities is None or df_activities.empty:
        print("No activity events in this window; skipping Spark/upsert.")
    else:
        df_activities = spark.createDataFrame(df_activities)
        #df_activities.show(1)
        # Upsert to Delta
        return_val = udf_UpsertDimension(
            df_activities, 
            dimensionType=1,
            targetPath=activity_path,
            naturalKeyColumnList=natural_key,
            primaryKeyColumnName=primary_key,
            flagSoftDeletes=False
        )
        print(return_val)

StatementMeta(, ab14784b-9cc0-4e5b-a91a-b1b073f0fd01, 311, Finished, Cancelled, Cancelled)

Fetching events from 2025-09-10T00:00:00.000Z to 2025-09-10T00:59:59.999Z
⚠️ Failed to parse JSON for URL: https://api.powerbi.com/v1.0/myorg/admin/activityevents?startDateTime='2025-09-10T00:00:00.000Z'&endDateTime='2025-09-10T00:59:59.999Z'
🔍 Status code: 400
📄 Raw response text: 
⚠️ Error in API call for 2025-09-10T00:00:00.000Z–2025-09-10T00:59:59.999Z: Expecting value: line 1 column 1 (char 0)
No activity events in this window; skipping Spark/upsert.
Fetching events from 2025-09-10T01:00:00.000Z to 2025-09-10T01:59:59.999Z
⚠️ Failed to parse JSON for URL: https://api.powerbi.com/v1.0/myorg/admin/activityevents?startDateTime='2025-09-10T01:00:00.000Z'&endDateTime='2025-09-10T01:59:59.999Z'
🔍 Status code: 400
📄 Raw response text: 
⚠️ Error in API call for 2025-09-10T01:00:00.000Z–2025-09-10T01:59:59.999Z: Expecting value: line 1 column 1 (char 0)
No activity events in this window; skipping Spark/upsert.
Fetching events from 2025-09-10T02:00:00.000Z to 2025-09-10T02:59:59.999Z
⚠️ Fai

Py4JJavaError: An error occurred while calling o140376.collectToPython.
: org.apache.spark.SparkException: Job 6560 cancelled part of cancelled job group 311
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:3086)
	at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:2956)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleJobGroupCancelled$4(DAGScheduler.scala:1247)
	at scala.runtime.java8.JFunction1$mcVI$sp.apply(JFunction1$mcVI$sp.java:23)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:79)
	at org.apache.spark.scheduler.DAGScheduler.handleJobGroupCancelled(DAGScheduler.scala:1246)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:3246)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:3224)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:3213)
	at org.apache.spark.util.Even